# Proyecto 3 - Visión por Computadora
## Integrantes:
- Mario Guerra - Carné: 21008
- Javier Alvarado - Carné: 21188
- Emilio Solano - Carné: 21212

### Inciso 1. Elección de imagen base.

In [ ]:
# Código aquí.

### Inciso 2. Detección de correspondencias y cálculo de homografías.

In [ ]:
# Código aquí.

### Inciso 3. Warping

In [ ]:
# Código aquí.

### Inciso 4. Blending

In [ ]:
# Código aquí.